In [9]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, clear_output

# Physical constants
mu_0 = (4 * np.pi) * 1e-7          # vacuum permeability [H/m]
d = 0.51054e-3                     # conductor diameter for AWG24 [m]

# Plotting limits (z in mm, B in Gauss)
z_min, z_max = -80, 13
B_min, B_max = -5, 10

# Sample window used for summary stats (mm)
z_top_sample, z_bottom_sample = -5, -60

# Common z-grid for curves (mm)
z_values = np.linspace(z_min, z_max, 1000)

# Reference offset (m) applied to external-field model
Homo_offset = 31.5e-3


In [10]:
# Axial field produced by a finite, multilayer solenoid (original closed form).
# Inputs accept scalars or arrays for z_mm. Result returned in Gauss.
def biot_savart(z_mm, R_mm, I_mA, N_turns, z_off_mm, layers):
    # Convert/alias once to keep algebra legible and control units explicitly
    z = z_mm / 1e3                  # mm -> m
    R = (R_mm * 2) / 1e3            # match original use of 2*R in metres
    I = I_mA / 1e3                  # mA -> A
    N = N_turns
    m = layers

    # Precompute linear terms and the shared core of the denominator
    num_plus  = (2 * ((z_off_mm/1e3) - z) + d * N)
    num_minus = (2 * ((z_off_mm/1e3) - z) - d * N)
    denom_core = 4 * (R + d/2 + (m - 1) * d)**2

    # Two symmetric contributions from the solenoid ends
    term_plus  = num_plus  / np.sqrt(denom_core + num_plus**2)
    term_minus = num_minus / np.sqrt(denom_core + num_minus**2)

    # Base expression in Tesla, then convert to Gauss
    B_z_T = mu_0 * I / (2 * d) * (term_plus - term_minus)
    return B_z_T * 1e4

# External field B(z) evaluated from an 8th-order polynomial.
# z is supplied in mm; internally shifted and converted to metres.
def compute_external_field(z_mm):
    z_m = z_mm / 1e3 + Homo_offset   # mm -> m and apply calibration shift
    Intercept = -0.05859
    B1 = 47.66114
    B2 = -4865.06371
    B3 = -146097.83504
    B4 = 4260040.3748
    B5 = 3.04437e8
    B6 = -8.75637e9
    B7 = -7.65903e10
    B8 = 1.46997e12
    return (Intercept + B1*z_m + B2*z_m**2 + B3*z_m**3 + B4*z_m**4 +
            B5*z_m**5 + B6*z_m**6 + B7*z_m**7 + B8*z_m**8)


In [11]:
# Label/size choices for a readable, aligned control panel
SLIDER_STYLE   = {'description_width': '120px'}
SLIDER_LAYOUT  = widgets.Layout(width='380px')
PICKER_LAYOUT  = widgets.Layout(width='220px')
GROUP_WIDTH    = '540px'

# Default colours to visually distinguish up to five coils
coil_colors = ['red', 'green', 'purple', 'orange', 'yellow']

# Build one parameter group per coil: [radius, turns, layers, z_offset, current, color]
coil_sliders = []
for i in range(5):
    s_radius  = widgets.FloatSlider(6.82,  min=1,   max=50,   step=0.1, description='Radius (mm):',
                                    style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True)
    s_turns   = widgets.IntSlider( 21,     min=1,   max=70,   step=1,   description='Turns:',
                                    style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True)
    s_layers  = widgets.IntSlider( 1,      min=1,   max=10,   step=1,   description='Layers:',
                                    style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True)
    s_zoff    = widgets.FloatSlider(0.0,   min=-100,max=0,    step=0.1, description='Z Offset (mm):',
                                    style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True)
    s_current = widgets.FloatSlider(0.0,   min=-1500,max=1500,step=0.1, description='Current (mA):',
                                    style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True)
    s_color   = widgets.ColorPicker(value=coil_colors[i], description='Color:',
                                    layout=PICKER_LAYOUT, style={'description_width':'60px'})
    coil_sliders.append([s_radius, s_turns, s_layers, s_zoff, s_current, s_color])

# Title + vertical grouping for each coil block
titles = [widgets.Label(value=f'Coil {i+1}', layout=widgets.Layout(margin='0 0 10px 0')) for i in range(5)]
coil_vboxes = [widgets.VBox([t] + sliders, layout=widgets.Layout(width=GROUP_WIDTH))
               for t, sliders in zip(titles, coil_sliders)]

# Single-column grid keeps labels from truncating and avoids horizontal crowding
slider_column = widgets.GridBox(coil_vboxes,
                                layout=widgets.Layout(grid_template_columns="repeat(1, 1fr)", width=GROUP_WIDTH))


In [12]:
# Ensure a clean state (no stray Axes from previous runs)
plt.close('all')

# Create four figures with explicit, non-auto display to control placement
with plt.ioff():
    fig_individual, ax_individual = plt.subplots(figsize=(7.0, 4.8))
    fig_sum,        ax_sum        = plt.subplots(figsize=(7.0, 4.8))
    fig_external,   ax_external   = plt.subplots(figsize=(7.0, 4.8))
    fig_all,        ax_all        = plt.subplots(figsize=(7.0, 4.8))

# Route only the canvases into widget outputs; keep figure objects in Python
out_individual = widgets.Output();  out_sum = widgets.Output()
out_external  = widgets.Output();  out_all = widgets.Output()

with out_individual: display(fig_individual.canvas)
with out_sum:        display(fig_sum.canvas)
with out_external:   display(fig_external.canvas)
with out_all:        display(fig_all.canvas)

# Tab titles make it easy to switch views during tuning
tabs = widgets.Tab(children=[out_individual, out_sum, out_external, out_all])
for i, name in enumerate(["Individual coils", "Sum of coils", "External field", "All fields"]):
    tabs.set_title(i, name)

# Compose full UI: controls on the left, tabs on the right
right = widgets.VBox([tabs], layout=widgets.Layout(flex='1 1 auto'))
ui = widgets.HBox([slider_column, right], layout=widgets.Layout(align_items='flex-start', gap='16px'))
display(ui)


In [13]:
# Snapshot current widget values to a structured list for plotting/metrics
def collect_coil_data():
    data = []
    for s in coil_sliders:
        radius, turns, layers, z_off, current, color = s
        data.append({
            'radius':   radius.value,
            'turns':    turns.value,
            'layers':   layers.value,
            'z_offset': z_off.value,
            'current':  current.value,
            'color':    color.value,
        })
    return data

# Per-coil traces displayed on their own axes for quick sanity checking
def plot_individual(ax, coils, z_mm):
    ax.cla()
    for i, c in enumerate(coils):
        B = biot_savart(z_mm, c['radius'], c['current'], c['turns'], c['z_offset'], c['layers'])
        ax.plot(z_mm, B, color=c['color'], label=f"Coil {i+1}")
    ax.set_xlim(z_min, z_max); ax.set_ylim(B_min, B_max); ax.grid(True)
    ax.set_xlabel('z-axis (mm)'); ax.set_ylabel('Magnetic Field (Gauss)')
    ax.set_title('Individual Coil Magnetic Fields'); ax.legend()

# Aggregate response from all coils; useful for matching/cancellation tasks
def plot_sum(ax, coils, z_mm):
    ax.cla()
    B_sum = np.zeros_like(z_mm, dtype=float)
    for c in coils:
        B_sum += biot_savart(z_mm, c['radius'], c['current'], c['turns'], c['z_offset'], c['layers'])
    ax.plot(z_mm, B_sum, color='black', label='Sum of coils')
    ax.set_xlim(z_min, z_max); ax.set_ylim(B_min, B_max); ax.grid(True)
    ax.set_xlabel('z-axis (mm)'); ax.set_ylabel('Magnetic Field (Gauss)')
    ax.set_title('Total Field Produced by Coils'); ax.legend()
    return B_sum

# External reference shown standalone for quick comparison of scales/shape
def plot_external(ax, z_mm):
    ax.cla()
    B_ext = compute_external_field(z_mm)
    ax.plot(z_mm, B_ext, color='pink', label='External field')
    ax.set_xlim(z_min, z_max); ax.set_ylim(B_min, B_max); ax.grid(True)
    ax.set_xlabel('z-axis (mm)'); ax.set_ylabel('Magnetic Field (Gauss)')
    ax.set_title('External Magnetic Field along z-axis'); ax.legend()
    return B_ext

# Overlay of coil-only, external, and combined fields, plus sample window markers
def plot_all(ax, coils, z_mm):
    ax.cla()
    B_ext = compute_external_field(z_mm)
    B_sum = np.zeros_like(z_mm, dtype=float)
    for c in coils:
        B_sum += biot_savart(z_mm, c['radius'], c['current'], c['turns'], c['z_offset'], c['layers'])
    B_total = B_sum + B_ext
    ax.plot(z_mm, B_sum,   color='orange', label='Sum of coils')
    ax.plot(z_mm, B_total, color='green',  linestyle='--', label='Total (coils + external)')
    ax.plot(z_mm, B_ext,   color='blue',   label='External')
    ax.axvline(x=z_bottom_sample, linestyle='--', color='red', label=f'Bottom: {z_bottom_sample} mm')
    ax.axvline(x=z_top_sample,    linestyle='--', color='red', label=f'Top: {z_top_sample} mm')
    ax.set_xlim(z_min, z_max); ax.set_ylim(B_min, B_max); ax.grid(True)
    ax.set_xlabel('z-axis (mm)'); ax.set_ylabel('Magnetic Field (Gauss)')
    ax.set_title('Optimised Design Corrected Field'); ax.legend()


In [14]:
# Single redraw routine to refresh all canvases in response to any control change
def redraw(*_):
    coils = collect_coil_data()
    plot_individual(ax_individual, coils, z_values); fig_individual.canvas.draw_idle()
    plot_sum(ax_sum, coils, z_values);               fig_sum.canvas.draw_idle()
    plot_external(ax_external, z_values);            fig_external.canvas.draw_idle()
    plot_all(ax_all, coils, z_values);               fig_all.canvas.draw_idle()

# Link every widget in every coil group to the redraw function
for group in coil_sliders:
    for w in group:
        w.observe(redraw, names='value')

# Initial draw so the UI is populated on load
redraw()


In [15]:
# Compute summary metrics of the combined field inside the sample interval
def calculate_field_statistics(coils):
    B_sum = np.zeros_like(z_values, dtype=float)
    for c in coils:
        B_sum += biot_savart(z_values, c['radius'], c['current'], c['turns'], c['z_offset'], c['layers'])
    B_total = B_sum + compute_external_field(z_values)

    mask = (z_values >= z_bottom_sample) & (z_values <= z_top_sample)
    B_total_sample = B_total[mask]

    mean_value    = float(np.mean(B_total_sample))
    peak_negative = float(np.min(B_total_sample))
    peak_positive = float(np.max(B_total_sample))
    return mean_value, peak_negative, peak_positive

# Button + output area to trigger/print fresh metrics on demand
button_calculate = widgets.Button(description="Refresh stats")
output = widgets.Output()
display(button_calculate, output)

def on_button_clicked(_):
    with output:
        clear_output(wait=True)
        coils = collect_coil_data()
        mean_v, neg_v, pos_v = calculate_field_statistics(coils)
        print(f"Mean value in sample range: {mean_v:.2f} G")
        print(f"Peak negative in sample range: {neg_v:.2f} G")
        print(f"Peak positive in sample range: {pos_v:.2f} G")

button_calculate.on_click(on_button_clicked)


Button(description='Refresh stats', style=ButtonStyle())

Output()